In [5]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure

In [6]:
# change style for my own use
from typing import Mapping, Tuple

from mediapipe.python.solutions import face_mesh_connections
from mediapipe.python.solutions import hands_connections
from mediapipe.python.solutions.drawing_utils import DrawingSpec
from mediapipe.python.solutions.hands import HandLandmark
from mediapipe.python.solutions.pose import PoseLandmark

_RADIUS = 16
_RED = (48, 48, 255)
_GREEN = (48, 255, 48)
_BLUE = (192, 101, 21)
_YELLOW = (0, 204, 255)
_GRAY = (128, 128, 128)
_PURPLE = (128, 64, 128)
_PEACH = (180, 229, 255)
_WHITE = (224, 224, 224)

# Hands
_THICKNESS_WRIST_MCP = 10
_THICKNESS_FINGER = 8
_THICKNESS_DOT = -1

# Hand landmarks
_PALM_LANMARKS = (HandLandmark.WRIST, HandLandmark.THUMB_CMC,
                  HandLandmark.INDEX_FINGER_MCP, HandLandmark.MIDDLE_FINGER_MCP,
                  HandLandmark.RING_FINGER_MCP, HandLandmark.PINKY_MCP)
_THUMP_LANDMARKS = (HandLandmark.THUMB_MCP, HandLandmark.THUMB_IP,
                    HandLandmark.THUMB_TIP)
_INDEX_FINGER_LANDMARKS = (HandLandmark.INDEX_FINGER_PIP,
                           HandLandmark.INDEX_FINGER_DIP,
                           HandLandmark.INDEX_FINGER_TIP)
_MIDDLE_FINGER_LANDMARKS = (HandLandmark.MIDDLE_FINGER_PIP,
                            HandLandmark.MIDDLE_FINGER_DIP,
                            HandLandmark.MIDDLE_FINGER_TIP)
_RING_FINGER_LANDMARKS = (HandLandmark.RING_FINGER_PIP,
                          HandLandmark.RING_FINGER_DIP,
                          HandLandmark.RING_FINGER_TIP)
_PINKY_FINGER_LANDMARKS = (HandLandmark.PINKY_PIP, HandLandmark.PINKY_DIP,
                           HandLandmark.PINKY_TIP)
_HAND_LANDMARK_STYLE = {
    _PALM_LANMARKS:
        DrawingSpec(
            color=_RED, thickness=_THICKNESS_DOT, circle_radius=_RADIUS),
    _THUMP_LANDMARKS:
        DrawingSpec(
            color=_PEACH, thickness=_THICKNESS_DOT, circle_radius=_RADIUS),
    _INDEX_FINGER_LANDMARKS:
        DrawingSpec(
            color=_PURPLE, thickness=_THICKNESS_DOT, circle_radius=_RADIUS),
    _MIDDLE_FINGER_LANDMARKS:
        DrawingSpec(
            color=_YELLOW, thickness=_THICKNESS_DOT, circle_radius=_RADIUS),
    _RING_FINGER_LANDMARKS:
        DrawingSpec(
            color=_GREEN, thickness=_THICKNESS_DOT, circle_radius=_RADIUS),
    _PINKY_FINGER_LANDMARKS:
        DrawingSpec(
            color=_BLUE, thickness=_THICKNESS_DOT, circle_radius=_RADIUS),
}

# Hands connections
_HAND_CONNECTION_STYLE = {
    hands_connections.HAND_PALM_CONNECTIONS:
        DrawingSpec(color=_GRAY, thickness=_THICKNESS_WRIST_MCP),
    hands_connections.HAND_THUMB_CONNECTIONS:
        DrawingSpec(color=_PEACH, thickness=_THICKNESS_FINGER),
    hands_connections.HAND_INDEX_FINGER_CONNECTIONS:
        DrawingSpec(color=_PURPLE, thickness=_THICKNESS_FINGER),
    hands_connections.HAND_MIDDLE_FINGER_CONNECTIONS:
        DrawingSpec(color=_YELLOW, thickness=_THICKNESS_FINGER),
    hands_connections.HAND_RING_FINGER_CONNECTIONS:
        DrawingSpec(color=_GREEN, thickness=_THICKNESS_FINGER),
    hands_connections.HAND_PINKY_FINGER_CONNECTIONS:
        DrawingSpec(color=_BLUE, thickness=_THICKNESS_FINGER)
}

# FaceMesh connections
_THICKNESS_TESSELATION = 1
_THICKNESS_CONTOURS = 2
_FACEMESH_CONTOURS_CONNECTION_STYLE = {
    face_mesh_connections.FACEMESH_LIPS:
        DrawingSpec(color=_WHITE, thickness=_THICKNESS_CONTOURS),
    face_mesh_connections.FACEMESH_LEFT_EYE:
        DrawingSpec(color=_GREEN, thickness=_THICKNESS_CONTOURS),
    face_mesh_connections.FACEMESH_LEFT_EYEBROW:
        DrawingSpec(color=_GREEN, thickness=_THICKNESS_CONTOURS),
    face_mesh_connections.FACEMESH_RIGHT_EYE:
        DrawingSpec(color=_RED, thickness=_THICKNESS_CONTOURS),
    face_mesh_connections.FACEMESH_RIGHT_EYEBROW:
        DrawingSpec(color=_RED, thickness=_THICKNESS_CONTOURS),
    face_mesh_connections.FACEMESH_FACE_OVAL:
        DrawingSpec(color=_WHITE, thickness=_THICKNESS_CONTOURS)
}

# Pose
_THICKNESS_POSE_LANDMARKS = 2
_POSE_LANDMARKS_LEFT = frozenset([
    PoseLandmark.LEFT_EYE_INNER, PoseLandmark.LEFT_EYE,
    PoseLandmark.LEFT_EYE_OUTER, PoseLandmark.LEFT_EAR, PoseLandmark.MOUTH_LEFT,
    PoseLandmark.LEFT_SHOULDER, PoseLandmark.LEFT_ELBOW,
    PoseLandmark.LEFT_WRIST, PoseLandmark.LEFT_PINKY, PoseLandmark.LEFT_INDEX,
    PoseLandmark.LEFT_THUMB, PoseLandmark.LEFT_HIP, PoseLandmark.LEFT_KNEE,
    PoseLandmark.LEFT_ANKLE, PoseLandmark.LEFT_HEEL,
    PoseLandmark.LEFT_FOOT_INDEX
])

_POSE_LANDMARKS_RIGHT = frozenset([
    PoseLandmark.RIGHT_EYE_INNER, PoseLandmark.RIGHT_EYE,
    PoseLandmark.RIGHT_EYE_OUTER, PoseLandmark.RIGHT_EAR,
    PoseLandmark.MOUTH_RIGHT, PoseLandmark.RIGHT_SHOULDER,
    PoseLandmark.RIGHT_ELBOW, PoseLandmark.RIGHT_WRIST,
    PoseLandmark.RIGHT_PINKY, PoseLandmark.RIGHT_INDEX,
    PoseLandmark.RIGHT_THUMB, PoseLandmark.RIGHT_HIP, PoseLandmark.RIGHT_KNEE,
    PoseLandmark.RIGHT_ANKLE, PoseLandmark.RIGHT_HEEL,
    PoseLandmark.RIGHT_FOOT_INDEX
])


def get_default_hand_landmarks_style() -> Mapping[int, DrawingSpec]:
  """Returns the default hand landmarks drawing style.
  Returns:
      A mapping from each hand landmark to its default drawing spec.
  """
  hand_landmark_style = {}
  for k, v in _HAND_LANDMARK_STYLE.items():
    for landmark in k:
      hand_landmark_style[landmark] = v
  return hand_landmark_style


def get_default_hand_connections_style(
) -> Mapping[Tuple[int, int], DrawingSpec]:
  """Returns the default hand connections drawing style.
  Returns:
      A mapping from each hand connection to its default drawing spec.
  """
  hand_connection_style = {}
  for k, v in _HAND_CONNECTION_STYLE.items():
    for connection in k:
      hand_connection_style[connection] = v
  return hand_connection_style


def get_default_face_mesh_contours_style(
) -> Mapping[Tuple[int, int], DrawingSpec]:
  """Returns the default face mesh contours drawing style.
  Returns:
      A mapping from each face mesh contours connection to its default drawing
      spec.
  """
  face_mesh_contours_connection_style = {}
  for k, v in _FACEMESH_CONTOURS_CONNECTION_STYLE.items():
    for connection in k:
      face_mesh_contours_connection_style[connection] = v
  return face_mesh_contours_connection_style


def get_default_face_mesh_tesselation_style() -> DrawingSpec:
  """Returns the default face mesh tesselation drawing style.
  Returns:
      A DrawingSpec.
  """
  return DrawingSpec(color=_GRAY, thickness=_THICKNESS_TESSELATION)


def get_default_face_mesh_iris_connections_style(
) -> Mapping[Tuple[int, int], DrawingSpec]:
  """Returns the default face mesh iris connections drawing style.
  Returns:
       A mapping from each iris connection to its default drawing spec.
  """
  face_mesh_iris_connections_style = {}
  left_spec = DrawingSpec(color=_GREEN, thickness=_THICKNESS_CONTOURS)
  for connection in face_mesh_connections.FACEMESH_LEFT_IRIS:
    face_mesh_iris_connections_style[connection] = left_spec
  right_spec = DrawingSpec(color=_RED, thickness=_THICKNESS_CONTOURS)
  for connection in face_mesh_connections.FACEMESH_RIGHT_IRIS:
    face_mesh_iris_connections_style[connection] = right_spec
  return face_mesh_iris_connections_style


def get_default_pose_landmarks_style() -> Mapping[int, DrawingSpec]:
  """Returns the default pose landmarks drawing style.
  Returns:
      A mapping from each pose landmark to its default drawing spec.
  """
  pose_landmark_style = {}
  left_spec = DrawingSpec(
      color=(0, 138, 255), thickness=_THICKNESS_POSE_LANDMARKS)
  right_spec = DrawingSpec(
      color=(231, 217, 0), thickness=_THICKNESS_POSE_LANDMARKS)
  for landmark in _POSE_LANDMARKS_LEFT:
    pose_landmark_style[landmark] = left_spec
  for landmark in _POSE_LANDMARKS_RIGHT:
    pose_landmark_style[landmark] = right_spec
  pose_landmark_style[PoseLandmark.NOSE] = DrawingSpec(
      color=_WHITE, thickness=_THICKNESS_POSE_LANDMARKS)
  return pose_landmark_style


In [3]:
# 计算2d三角形面积公式
def heron(a,b,c):  
    s = (a + b + c) / 2   
    area = (s*(s-a) * (s-b)*(s-c)) ** 0.5        
    return area

def distance2d(x1,y1,z1,x2,y2,z2):    
    a=(x1-x2)**2+(y1-y2)**2
    d= a ** 0.5  
    return d  

def areatriangle2d(x1,y1,z1,x2,y2,z2,x3,y3,z3):  
    a=distance2d(x1,y1,z1,x2,y2,z2)  
    b=distance2d(x2,y2,z2,x3,y3,z3)  
    c=distance2d(x3,y3,z3,x1,y1,z1)  
    A = heron(a,b,c)  
    print("area of triangle is %r " %A)
    
def areatriangle2d_By_Points(p1,p2,p3,w,h):  
    x1 = p1.x*w
    y1 = p1.y*h
    z1 = p1.z*w
    
    x2 = p2.x*w
    y2 = p2.y*h
    z2 = p2.z*w
    
    x3 = p3.x*w
    y3 = p3.y*h
    z3 = p3.z*w
    
#     z1 = 0
#     z2 = 0
#     z3 = 0
    a=distance2d(x1,y1,z1,x2,y2,z2)  
    b=distance2d(x2,y2,z2,x3,y3,z3)  
    c=distance2d(x3,y3,z3,x1,y1,z1)  
    A = heron(a,b,c)
    return A


def areatriangle2d_sum(hand_landmarks,w,h): 
    extract_point = [4,8,12,16,20,0]
    p = []
    for i in extract_point:
        p.append(hand_landmarks.landmark[i])
    a1 = areatriangle2d_By_Points(p[0],p[1],p[2],w,h)
    a2 = areatriangle2d_By_Points(p[0],p[2],p[3],w,h)
    a3 = areatriangle2d_By_Points(p[0],p[3],p[4],w,h)
    a4 = areatriangle2d_By_Points(p[0],p[4],p[5],w,h)
    
    a = a1+a2+a3+a4
    return a


def draw_plot(hand_area,hand_z,hand_landmarks,image_width,image_height):
#     clear_output(wait=True)
#         plt.cla()
        # clear axis
        ax1.cla()
        ax2.cla()
        hand_area.popleft()
        hand_z.popleft()
        area = areatriangle3d_sum(hand_landmarks,image_width,image_height)
        hand_area.append(area)
        hand_z.append(hand_landmarks.landmark[4].z*image_width)
        
#         plt.subplot(121)
        ax1.plot(hand_area)
#         ax1.title("Hand area tracking with time")
#         ax1.xlabel("Time")
#         ax1.ylabel("Hand area")
        
#         plt.subplot(122)
        ax2.plot(hand_z)
#         ax2.title("Hand thumb deep change with time")
#         ax2.xlabel("Time")
#         ax2.ylabel("Hand Thumb deep length z")

In [8]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

IMAGE_FILES = []

img_path2 = "data/"
IMAGE_FILES.append(img_path2+'/11.jpeg')
IMAGE_FILES.append(img_path2+'/22.jpeg')
IMAGE_FILES.append(img_path2+'/33.jpeg')
IMAGE_FILES.append(img_path2+'/44.jpeg')
IMAGE_FILES.append(img_path2+'/55.jpeg')
IMAGE_FILES.append(img_path2+'/66.jpeg')
print(len(IMAGE_FILES))
print(IMAGE_FILES)
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.0001) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.flip(cv2.imread(file), 1)
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if not results.multi_hand_landmarks:
      print('False'+str(idx))
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
       
      area = areatriangle2d_sum(hand_landmarks,image_width,image_height)
      print(area)
      
      print(idx)
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          get_default_hand_landmarks_style(),
          get_default_hand_connections_style())
      cv2.imwrite(str(idx+1) + str(idx+1)+ 'out.jpg', cv2.flip(annotated_image, 1))

6
['data//11.jpeg', 'data//22.jpeg', 'data//33.jpeg', 'data//44.jpeg', 'data//55.jpeg', 'data//66.jpeg']
272305.8826097307
0
340710.8440165698
1
60820.24408519206
2
185295.22618299306
3
77194.37558109598
4
266416.54404802556
5


0=>left
1=>right
5=>left mirror